In [55]:
#Import libraries and functions

%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import glob #pattern package to find filename
from skimage.io import imread
from scipy import ndimage
from scipy.ndimage import shift
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage.interpolation import rotate
from scipy.optimize import curve_fit
from pymatgen.io.phonopy import get_ph_bs_symm_line

In [86]:
#Create triangular mesh

##Define reciprocal space directions to generate q-mesh (see notes)

a = np.array([0, 0, 0]) 
b = np.array([-4/9, -1/9, 5/9]) #this is (-1/3, 2/3) in hexagonal basis
c = np.array([5/9, -1/9, -4/9]) #this is (2/3, 1/3) in hexagonal basis

N = 161

kmesh = np.zeros([N**2, 5])

index = 0


# ##This creates a triangular mesh

# for i in range(0, N):
#     for j in range(0, N):
#         if i + j <= N-1:
#             s = a + (b - a)*i/N + (c - a)*j/N
#             kmesh[index] = [i, j, s[0], s[1], s[2]]
#             index += 1

# # print(kmesh)
# kmesh = kmesh[~np.all(kmesh == 0, axis=1)] #removes all array elements that are zero
# # print(kmesh)
# # kmesh = np.around(kmesh, 3)

##This creates a parallelogram mesh

for i in range(0, N):
    for j in range(0, N):
        s = a + (b - a)*i/N + (c - a)*j/N
        kmesh[index] = [i, j, s[0], s[1], s[2]]
        index += 1

##Sort q values so that it forms a connected path in q space: first goes from a to c, and then coils back

for i in range(0, N):
    temp = kmesh[kmesh[:,0] == i]
    if i & 1:        
        ind = np.argsort(-1*temp[:,1])
        temp = temp[ind]
        kmesh[kmesh[:,0] == i] = temp
    else:
        ind = np.argsort(temp[:,1])
        temp = temp[ind]
        kmesh[kmesh[:,0] == i] = temp

# print(kmesh)

In [87]:
#transforming to hexagonal setting to check

tr_mat = np.array([[1, -1, 0], [0, 1, -1], [1, 1, 1]])
kmesh_hex = np.transpose(np.matmul(tr_mat, np.transpose(kmesh[:,2:5])))

# k_hex = np.transpose(np.matmul(tr_mat, np.transpose([1/9, -2/9, 1/9])))
                     
# print(k_hex)
# print(kmesh_hex)

In [88]:
#Transforming to Cartesian setting to check

#Material-specific lattice parameters (in angstrom)

a = 4.29
c = 40.56

lattice_para = np.array([[a*np.cos(0), a*np.sin(0), 0], \
                         [a*np.cos(2*np.pi/3), a*np.sin(2*np.pi/3), 0], \
                         [0, 0, c]])

ra = 4*np.pi/(np.sqrt(3)*a)
rc = 2*np.pi/c

rec_lattice_para = np.array([[ra*np.cos(np.pi/6), ra*np.sin(np.pi/6), 0], \
                             [ra*np.cos(5*np.pi/6), ra*np.sin(5*np.pi/6), 0], \
                             [0, 0, rc]])
def Q(h, k, l):
    return h*rec_lattice_para[0] + k*rec_lattice_para[1] + l*rec_lattice_para[2]

###converting from hexagonal to cartesian

qmesh_cart = kmesh_hex*0

for i in range(0, np.shape(qmesh_cart)[0]):
    qmesh_cart[i] = Q(kmesh_hex[i, 0], kmesh_hex[i, 1], kmesh_hex[i, 2])
    
# print(qmesh_cart)

In [89]:
path = 'C:/Users/Hari/OneDrive/Research/Ultrafast Optics/MnBi2Te4/phon_Hari/'

np.savetxt(path + 'python_qpoints.txt', kmesh[:,2:5])  

In [90]:
path = 'C:/Users/Hari/OneDrive/Research/Ultrafast Optics/SLAC/sim-phonon-struct/'

np.savetxt(path + 'python_qpoints.txt', kmesh[:,2:5])  

In [91]:
path = 'C:/Users/Hari/OneDrive/Research/Ultrafast Optics/SLAC/sim-phonon-struct/'

np.savetxt(path + 'python_qpoints_hex.txt', kmesh_hex)  

In [40]:
from pymatgen.io.phonopy import get_ph_bs_symm_line

path = 'C:/Users/Hari/OneDrive/Research/Ultrafast Optics/MnBi2Te4/phon_Hari/'

a = get_ph_bs_symm_line(path + "qpoints.yaml")

In [47]:
a.band_reorder()     

In [48]:
print(np.shape(a.eigendisplacements))

print(np.real(a.eigendisplacements[3,2,1]))

print(np.real(a.eigendisplacements[3,82,1]))

print(np.real(a.bands[3,2]))
print(np.real(a.bands[3,82]))



(21, 1681, 7, 3)
[4.38164543e-04 2.47342464e-03 3.82413274e-06]
[-1.92296630e-03 -1.61617395e-03  3.82413274e-06]
0.9374873352
0.9374873352


In [20]:
hex_to_rh = 1/3*np.array([[2, 1, 1], [-1, 1, 1], [-1, -2, 1]])
hkl_hex = np.array([-1/3, -2/3, 0])

hkl_rh = np.matmul(hex_to_rh, np.transpose(hkl_hex))

print(hkl_rh)

[-0.44444444 -0.11111111  0.55555556]
